#### 7/1　選定したringで学習

In [1]:
import pandas as pd
from math import sqrt as sqrt
from itertools import product as product

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Function
from torchsummary import summary
import torch.utils.data as data
import torch.optim as optim

import ast
import cv2
import numpy as np
import time
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

In [2]:
from utils.ssd_model import SSD
from utils.ssd_model import MultiBoxLoss

In [4]:
input_size = 300
color_mean = (0, 0)
voc_classes = ['ring']

In [5]:
ssd_cfg = {
    'num_classes': 2,  # 背景クラスを含めた合計クラス数
    'input_size': input_size,  # 画像の入力サイズ
    'bbox_aspect_num': [4, 6, 6, 6, 4, 4],  # 出力するDBoxのアスペクト比の種類
    'feature_maps': [38, 19, 10, 5, 3, 1],  # 各sourceの画像サイズ   
#     'feature_maps': [38, 19, 10, 5, 3, 1],  # 各sourceの画像サイズ
    'steps': [8, 16, 32, 64, 100, 300],  # DBOXの大きさを決める
    'min_sizes': [30, 60, 111, 162, 213, 264],  # DBOXの大きさを決める
    'max_sizes': [60, 111, 162, 213, 264, 315],  # DBOXの大きさを決める
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}

### parameterの読み込み

In [ ]:
net_w = SSD(phase='inference', cfg=ssd_cfg)
net_weights = torch.load('parameterのパス',
                        map_location={'cuda:0': 'cpu'})
net_w.load_state_dict(net_weights)

In [ ]:
ssd = SSDPredictShow(eval_categories=voc_classes, net=net_w)
ssd.show('パス', data_confidence_level=0.9)
plt.show()